In [1]:

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

from models.text_rnn import TextRNN
from models.text_cnn import TextCNN



In [10]:
max_features = 50
maxlen = 100
batch_size = 16
embedding_dims = 20
epochs = 1

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')



Loading data...
25000 train sequences
25000 test sequences


In [14]:
y_train[1:20]

array([0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1])

In [12]:
print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
x_train[1]


Pad sequences (samples x time)...
x_train shape: (25000, 100)
x_test shape: (25000, 100)


array([ 2, 11,  2,  2,  4,  2,  9,  2,  2,  7,  2,  2,  2,  2,  2,  2,  2,
        2, 15,  2,  2,  2,  2,  2, 15,  2,  2,  2,  2,  5,  4,  2,  9, 43,
        2,  2, 15,  2,  2,  5,  2,  2, 11,  2,  2,  2,  2,  9,  2,  2,  2,
        5,  2,  2,  2,  2,  5,  4,  2,  2,  2,  2, 18,  2, 32,  2,  2, 14,
        9,  6,  2,  2, 22,  2,  2,  2,  9,  8,  2,  2, 23,  4,  2, 15, 16,
        4,  2,  5, 28,  6,  2,  2,  2, 33,  2,  2,  2, 16,  2,  2],
      dtype=int32)

In [ ]:
print('Build model...')
model = TextCNN(maxlen, max_features, embedding_dims)
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])




In [ ]:
print('Train...')
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))



In [15]:
print('Test...')
result = model.predict(x_test)

Test...


ValueError: The maxlen of inputs of TextCNN must be 40, but now is 100

In [21]:
import tensorflow_hub as hub
import tensorflow as tf

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-lite/2")
#embeddings = embed(["cat is on the mat", "dog is in the fog"])

In [25]:
embed.signatures['default'](dense_shape=[13,], values='batch', indices=12)

ValueError: All inputs to `ConcreteFunction`s must be Tensors; on invocation of pruned, the 0-th input ([13]) was not a Tensor.

In [22]:
embeddings = embed(["Cat is on the mat", "dog is in the fog"])

TypeError: 'AutoTrackable' object is not callable

In [16]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-lite/2", output_shape=[50],
                           input_shape=[], dtype=tf.string)

model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

TypeError: in converted code:

    /home/wroscoe/code/gradest/venv/lib/python3.6/site-packages/tensorflow_hub/keras_layer.py:209 call  *
        result = f()
    /home/wroscoe/code/gradest/venv/lib/python3.6/site-packages/tensorflow_core/python/eager/function.py:1551 __call__  *
        return self._call_impl(args, kwargs)
    /home/wroscoe/code/gradest/venv/lib/python3.6/site-packages/tensorflow_core/python/eager/function.py:1570 _call_impl
        ).format(self._num_positional_args, self._arg_keywords, args))

    TypeError: Expected at most 0 positional arguments (and the rest keywords, of ['values', 'indices', 'dense_shape']), got (<tf.Tensor 'keras_layer_2_input:0' shape=(None,) dtype=string>,). When calling a concrete function, positional arguments may not be bound to Tensors within nested structures.


In [8]:

import tensorflowjs as tfjs

In [11]:
tfjs.converters.save_keras_model(model, './js_model')

In [17]:
!pip3 install sentencepiece

import sentencepiece

    100% |████████████████████████████████| 1.0MB 5.6MB/s ta 0:00:011


In [19]:
!pip3 install --upgrade tensorflow

    100% |████████████████████████████████| 421.8MB 70kB/s  eta 0:00:01  9% |███                             | 39.1MB 17.7MB/s eta 0:00:22    17% |█████▍                          | 71.7MB 37.4MB/s eta 0:00:10    19% |██████▍                         | 84.2MB 3.7MB/s eta 0:01:31    34% |███████████                     | 146.0MB 21.7MB/s eta 0:00:13    36% |███████████▋                    | 152.4MB 28.0MB/s eta 0:00:10    37% |████████████▏                   | 160.1MB 12.1MB/s eta 0:00:22    42% |█████████████▊                  | 180.3MB 11.9MB/s eta 0:00:21    46% |██████████████▊                 | 194.4MB 5.0MB/s eta 0:00:46    47% |███████████████▎                | 201.9MB 12.8MB/s eta 0:00:18    53% |█████████████████               | 225.2MB 14.1MB/s eta 0:00:14    70% |██████████████████████▍         | 295.7MB 13.3MB/s eta 0:00:10    78% |█████████████████████████▎      | 332.8MB 5.5MB/s eta 0:00:17    96% |███████████████████████████████ | 408.3MB 16.2MB/s eta 0:00:01
  Found existi

In [20]:
with tf.Session() as sess:
  module = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-lite/2")
  spm_path = sess.run(module(signature="spm_path"))
  # spm_path now contains a path to the SentencePiece model stored inside the
  # TF-Hub module

sp = spm.SentencePieceProcessor()
sp.Load(spm_path)

AttributeError: module 'tensorflow' has no attribute 'Session'

In [26]:
import tensorflow as tf

In [39]:
export_dir = '/home/wroscoe/code/use_lite_tfjs/web_model'
import os
[i for i in os.walk(export_dir)]

[('/home/wroscoe/code/use_lite_tfjs/web_model',
  [],
  ['group1-shard4of7',
   'group1-shard7of7',
   'group1-shard2of7',
   'weights_manifest.json',
   'group1-shard5of7',
   'tensorflowjs_model.pb',
   'group1-shard6of7',
   'group1-shard1of7',
   'group1-shard3of7'])]

In [40]:

tf.saved_model.load(
    export_dir, tags=None
)

OSError: SavedModel file does not exist at: /home/wroscoe/code/use_lite_tfjs/web_model/{saved_model.pbtxt|saved_model.pb}